In [ ]:
#| default_exp config

# EVCI config

> **API**: The inputs are specified through 4 excel sheets for frieght corridor analysis. The python code reads these inputs parameters from the following excel sheets for analysis.

1. model.xlsx - This contains all the global parameters that are model specific. They remain valid for every corridor that needs analysis.
2. sites.xlsx - This contains a list of sites, with their latitude and longitude for analysis. This is an initial list of sites. This file needs to be created for each freight corridor
3. traffic.xlsx - This contains a typical traffic profile around the sites for each freight corridor.
4. grid.xlsx - This contains information about neighboring distribution transformers from where the chargers deployed at each site will draw power from.

In [ ]:
#|hide
from nbdev.showdoc import *

# Read input data

In [ ]:
#|export

import os
import numpy as np
import pandas as pd
import json


def setup_and_read_data(corridor:str, input_path='input/', output_path='output/'):
    "This function sets up paths and reads input excel files for a specified corridor"

    INPUT_PATH = input_path + corridor + '/'
    OUTPUT_PATH = output_path + corridor + '/'

    if not os.path.exists(OUTPUT_PATH):
        os.mkdir (output_path + corridor)
        
    model   = pd.read_excel(input_path + "model.xlsx",sheet_name=['charger_specific','battery_specific','others'])
    sites   = pd.read_excel(INPUT_PATH + "sites.xlsx",sheet_name=['sites'])
    traffic = pd.read_excel(INPUT_PATH + "traffic.xlsx",sheet_name=['profile'])
    grid    = pd.read_excel(INPUT_PATH + "grid.xlsx",sheet_name=['grid'])
    
    return model, sites, traffic, grid, INPUT_PATH, OUTPUT_PATH

`Arguments`:

1. `corridor`: a string that identifies the corridor being analyzed (e.g. chandigarh_leh)
2. `input_path`: a string denoting base directory under which input files (xlsx and shape files) are available for analysis. Default is `'input/'`
3. `output_path`: a string denoting base directory under which output files will be stored. Default is `'output/'`

`Returns`:

1. `model`: dataframe of model parameters (from model.xlsx)
2. `sites`: dataframe of sites (from sites.xlsx)
3. `traffic`: dataframe of traffic profile (from traffic.xlsx)
4. `grid`: dataframe of grid parameters (from grid.xlsx)
5. `INPUT_PATH`: a string indicating the input_path (e.g. input/chandigarh_leh/)
6. `OUTPUT_PATH`: a string indicating the output path (e.g. output/chandigarh_leh/)


In [ ]:
#example usage
m,s,t,g,INPUT_PATH,OUTPUT_PATH = setup_and_read_data('chandigarh_karnal')

# Data availability check

Let's check if the four excel sheets provided have the correctly named worksheets within them.

In [ ]:
#| export
model_sheets = ['charger_specific', 'battery_specific', 'others']
sites_sheets = ['sites']
traffic_sheets = ['profile']
grid_sheets = ['grid']

def data_availability_check(m,s,t,g): 
    "This function checks if the excel files contain the mandatory worksheets."
    
    retval = []
    
    if list(m.keys()) != model_sheets: retval.append('model')
    if list(s.keys()) != sites_sheets: retval.append('sites')
    if list(t.keys()) != traffic_sheets: retval.append('traffic')
    if list(g.keys()) != grid_sheets: retval.append('grid')
    
    #assert data_availability_check(list(m.keys()),model_sheets) == True, \
    #  f"model.xlsx must contain the sheets: {model_sheets}" 
    #assert data_availability_check(list(s.keys()),['sites']) == True, \
    #  f"sites.xlsx must contain the sheet {sites_sheets}" 
    #assert data_availability_check(list(t.keys()),['profile']) == True, 
    #  f"traffic.xlsx must contain the sheet {traffic_sheets}" 
    #assert data_availability_check(list(g.keys()),['grid']) == True, \
    #  f"grid.xlsx must contain the sheet {grid_sheets}"
    
    return retval

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)

`Returns`:

A list of xlsx file names wiht missing mandatory sheets

In [ ]:
data_availability_check(m,s,t,g)

[]

# Data integrity check

Let's now check if any of the mandatory columns in each of the worksheets are all empty!

In [ ]:
#| export
def data_integrity_check(m,s,t,g, verbose=False):
    "This function checks for integrity of excel data by checking missing values."
    missing = []
    
    for x in [m,s,t,g]:
        tmpx = {}
        for k in x.keys():
            total = x[k].shape[0]
            tmpx[k] = []
            for c in x[k].columns:
                if sum(pd.isna(x[k][c])) > 0:
                    if verbose: print(f"Column '{c}' of '{k}' has {sum(pd.isna(x[k][c]))}/{total} missing values")
                    tmpx[k].append(c)
        missing.append(tmpx)
                    
    return missing

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)

`Returns`:

A dictionary of missing columns with their corresponding xlsx filename and worksheet name.

In [ ]:
data_integrity_check(m,s,t,g)

[{'charger_specific': ['Range',
   'UoM',
   '2W',
   '3WS',
   '4WS',
   '4WF',
   'Unnamed: 9',
   '2W.1',
   '3WS.1',
   '4WS.1',
   '4WF.1'],
  'battery_specific': ['Parameter',
   'Value',
   'Range',
   'User Input?',
   'Unnamed: 4',
   'Same battery swapping station is used for 2W and 3W',
   'Unnamed: 6',
   'Unnamed: 7',
   'Unnamed: 8',
   'Unnamed: 9',
   'Unnamed: 10',
   'Unnamed: 11',
   'Unnamed: 12',
   'Unnamed: 13',
   'Unnamed: 14',
   'Unnamed: 15',
   'Unnamed: 16',
   'Unnamed: 17',
   'Unnamed: 18',
   'Unnamed: 19'],
  'others': ['Range']},
 {'sites': ['Address']},
 {'profile': []},
 {'grid': []}]

In [ ]:
# verbose output
data_integrity_check(m,s,t,g,verbose=True)

Column 'Range' of 'charger_specific' has 12/15 missing values
Column 'UoM' of 'charger_specific' has 8/15 missing values
Column '2W' of 'charger_specific' has 8/15 missing values
Column '3WS' of 'charger_specific' has 8/15 missing values
Column '4WS' of 'charger_specific' has 8/15 missing values
Column '4WF' of 'charger_specific' has 8/15 missing values
Column 'Unnamed: 9' of 'charger_specific' has 15/15 missing values
Column '2W.1' of 'charger_specific' has 9/15 missing values
Column '3WS.1' of 'charger_specific' has 9/15 missing values
Column '4WS.1' of 'charger_specific' has 9/15 missing values
Column '4WF.1' of 'charger_specific' has 9/15 missing values
Column 'Parameter' of 'battery_specific' has 10/18 missing values
Column 'Value' of 'battery_specific' has 10/18 missing values
Column 'Range' of 'battery_specific' has 18/18 missing values
Column 'User Input?' of 'battery_specific' has 10/18 missing values
Column 'Unnamed: 4' of 'battery_specific' has 18/18 missing values
Column 'S

[{'charger_specific': ['Range',
   'UoM',
   '2W',
   '3WS',
   '4WS',
   '4WF',
   'Unnamed: 9',
   '2W.1',
   '3WS.1',
   '4WS.1',
   '4WF.1'],
  'battery_specific': ['Parameter',
   'Value',
   'Range',
   'User Input?',
   'Unnamed: 4',
   'Same battery swapping station is used for 2W and 3W',
   'Unnamed: 6',
   'Unnamed: 7',
   'Unnamed: 8',
   'Unnamed: 9',
   'Unnamed: 10',
   'Unnamed: 11',
   'Unnamed: 12',
   'Unnamed: 13',
   'Unnamed: 14',
   'Unnamed: 15',
   'Unnamed: 16',
   'Unnamed: 17',
   'Unnamed: 18',
   'Unnamed: 19'],
  'others': ['Range']},
 {'sites': ['Address',
   'Traffic congestion',
   'Year for Site recommendation',
   'Hoarding/Kiosk (1 is yes & 0 is no)',
   'Hoarding margin',
   'Kiosk margin',
   'Available area',
   'Upfront cost per sqm (land)',
   'Yearly cost per sqm (land)',
   'Upfront cost per sqm (kiosk)',
   'Yearly cost per sqm (kiosk)',
   'Upfront cost per sqm (hoarding)',
   'Yearly cost per sqm (hoarding)',
   'Battery swap available']},

# Read global variables from xlsx

In [ ]:
#|export

def read_globals(m,s,t,g,ui_inputs):
    "This function returns all global parameters read from the xlsx."
    
    r = {}
    df_c = m['charger_specific']
    df_b = m['battery_specific']
    df_o = m['others']
    
    # read all other parameters from the xlsx
    
    r['M'] = df_c[df_c['Parameter']=='vehicle_types']['Value'].iloc[0].split(',')
    # or should it be from the UI having selected a subset for analysis?
    r['M'] = ui_inputs['M']
    r['C'] = df_c[df_c['Parameter']=='charger_types']['Value'].iloc[0].split(',')
    r['Kj'] = eval(df_c[df_c['Parameter']=='Kj']['Value'].iloc[0])
    r['Dj'] = eval(df_c[df_c['Parameter']=='Dj']['Value'].iloc[0])
    r['Hj'] = eval(df_c[df_c['Parameter']=='Hj']['Value'].iloc[0])
    r['Qj'] = eval(df_c[df_c['Parameter']=='Qj']['Value'].iloc[0])
    r['tj'] = eval(df_c[df_c['Parameter']=='tj']['Value'].iloc[0])
    r['Mj'] = eval(df_c[df_c['Parameter']=='Mj']['Value'].iloc[0])
    r['Gk'] = eval(df_c[df_c['Parameter']=='Gk']['Value'].iloc[0])

    r['N'] = 500
    r['Ng'] = 0

    r['timeslots'] = {k: 24/v for k, v in r['tj'].items()}
    timeslots = r['timeslots']
    
    r['Nc'] = s['sites'].shape[0]
    Nc = r['Nc']

    r['Gi'] = [0]*Nc
    r['di'] = [0]*Nc
    r['Wi'] = [0]*Nc
    r['Ri'] = [0]*Nc
    r['Ai'] = [50]*Nc
    r['Li'] = [1500]*Nc
    r['Bi'] = [0.25 * 3.5 * 24 * 365]*Nc # 25% of Rs 3.5/KWh per year

    r['Eg'] = {k: [5.5] * int(v) for k, v in timeslots.items()}
    r['Er'] = {k: [0] * int(v) for k, v in timeslots.items()}
    r['Mg'] = {k: [5.5 * 0.15] * int(v) for k, v in timeslots.items()}
    r['Mr'] = {k: [0] * int(v) for k, v in timeslots.items()}
    r['l']  = {k: [1] * int(v) for k, v in timeslots.items()}
    
    r['MH'] = [s['sites'].loc[i]['Hoarding margin'] for i in range(Nc)]
    r['MK'] = [0.15]*Nc

    #Traffic Model
    # read hourly vehicular traffic from the traffic.xlsx else use default values
    # peak vehicles through crowded junctions in a day ~ 1.5L

    peak_traffic = [
             4826, 4826, 5228, 5228, 5228, 5630, 6434, 6836, 6836, 
             6434, 6032, 6032, 6032, 6032, 6434, 6836, 7239, 8043, 
             8043, 8043, 6836, 6032, 5630, 5228       
    ]

    if 'profile' in t:
      r['peak_traffic'] = t['profile'].vehicles.tolist()[:24] 
      peak_traffic = r['peak_traffic']
    else:
      printf("Using default traffic profile of around 55000 vehicles per day")
      r['peak_traffic'] = peak_traffic

    # Average traffic approx 80% of peak
    avg_traffic = [i*.8 for i in peak_traffic]
    
    avg_traffic_3W = [i*.4 for i in avg_traffic]
    avg_traffic_4W = [i*.2 for i in avg_traffic]
  
    djworking_hourly_3WS = [i/5 for i in avg_traffic_3W]
    djworking_hourly = [i/5 for i in avg_traffic_4W]
    djworking_half_hourly = [val for val in djworking_hourly 
                             for _ in (0, 1)]
    djworking_one_and_half_hourly = list(np.mean(np.array(djworking_half_hourly).reshape(-1, 3), axis=1))
    r['djworking_hourly'] = djworking_hourly
    r['djworking_half_hourly'] = djworking_half_hourly
    r['djworking_one_and_half_hourly'] = djworking_one_and_half_hourly
    
    djworking = {}
    djworking['3WS'] = [np.round(i,0) for i in djworking_hourly_3WS]
    djworking['4WF'] = [np.round(i,0) for i in djworking_half_hourly]
    djworking['4WS'] = [np.round(i,0) for i in djworking_one_and_half_hourly]
    r['djworking'] = djworking        

    r['Cij'] = {'3WS':[1]*Nc, '4WS': [1]*Nc, '4WF':[1]*Nc}
    
    # now override the defaults with the read values from the UI parameters into r
    x = json.dumps(ui_inputs)
    ui_inputs = json.loads(x)
    
    r['K'] = ui_inputs['years_of_analysis']
    r['charger_types'] = ui_inputs['M']
    r['years_of_analysis'] = ui_inputs['years_of_analysis']
    r['capex_3WS'] = ui_inputs['capex_3WS']
    r['capex_4WS'] = ui_inputs['capex_4WS']
    r['capex_4WF'] = ui_inputs['capex_4WF']
    r['hoarding_cost'] = 900000
    r['kiosk_cost'] = 180000
    r['year1_conversion'] = ui_inputs['year1_conversion']
    r['year2_conversion'] = ui_inputs['year2_conversion']
    r['year3_conversion'] = ui_inputs['year3_conversion']
    r['fast_charging'] = ui_inputs['fast_charging']
    r['slow_charging'] = ui_inputs['slow_charging']
    r['holiday_percentage'] = ui_inputs['holiday_percentage']
    
    # now lets derive all other parameters that depend on the UI inputs.
    r['CH'] = [r['hoarding_cost']]*Nc
    r['CK'] = [r['kiosk_cost']]*Nc
    r['pj'] = {1: r['year1_conversion'], 
          2: r['year2_conversion'], 
          3: r['year3_conversion']}

    r['Pj'] = max(r['pj'].values()) 

    holiday_percentage = r['holiday_percentage']
    djholiday = {}
    djholiday['3WS'] = [np.round(i*holiday_percentage,0) for i in djworking_hourly_3WS]
    djholiday['4WF'] = [np.round(i*holiday_percentage,0) for i in djworking_half_hourly]
    djholiday['4WS'] = [np.round(i*holiday_percentage,0) for i in djworking_one_and_half_hourly]
    r['djholiday'] = djholiday
    
    fast_charging = float(df_o[df_o['Parameter']=='slow charger margin']['Value'].iloc[0])
    slow_charging = float(df_o[df_o['Parameter']=='fast charger margin']['Value'].iloc[0])

    r['qjworking'] = {'4WS': [slow_charging] * int(timeslots['4WS']), 
                 '4WF': [fast_charging] * int(timeslots['4WF']), 
                 '3WS': [fast_charging + slow_charging] * int(timeslots['3WS']), 
                  }
    r['qjholiday'] = {'4WS': [slow_charging] * int(timeslots['4WS']), 
                 '4WF': [fast_charging] * int(timeslots['4WF']), 
                 '3WS': [fast_charging + slow_charging] * int(timeslots['3WS']), 
                 }
    
    return r

`Arguments`:

1. `m`: dataframe of model parameters (from model.xlsx)
2. `s`: dataframe of sites (from sites.xlsx)
3. `t`: dataframe of traffic profile (from traffic.xlsx)
4. `g`: dataframe of grid parameters (from grid.xlsx)
5. `ui_inputs`: dictionary of all parameters selected by use from the frontend with the UI

`Returns`:

A dictionary with all hyperparameters required for the model to run

In [ ]:
# example usage

ui_inputs = { 
    "backoff_factor": 1,
    "M": ["3WS", "4WS", "4WF"],
    "years_of_analysis": [1,2,3],
    "capex_3WS": 112000,
    "capex_4WS": 250000,
    "capex_4WF": 1500000,
    "hoarding cost": 900000,
    "kiosk_cost": 180000,
    "year1_conversion": 0.02,
    "year2_conversion": 0.05,
    "year3_conversion": 0.1,
    "holiday_percentage": 0.3,
    "fast_charging": 0.3,
    "slow_charging": 0.15,
    "cluster": False,
    "cluster_th": 0.2,
    "plot_dendrogram": False,
    "use_defaults": False 
}

r = read_globals(m,s,t,g, ui_inputs)
print(r['Kj'])
print(r['charger_types'])

{'3WS': 112000, '4WS': 250000, '4WF': 1500000}
['3WS', '4WS', '4WF']
